# Sample shapes

Reconstruct shapes based on sinograms of sinograms of log(intensity)

- all peaks
- peaks assigned to a phase
- impurity of precipitate peaks

Uses the mlem reconstruction suggested by Bonnin et al, APL, 2014

JW March 2024

In [ ]:
import numpy as np, pylab as pl, h5py
import sys, os
sys.path.insert(0, os.environ['HOME'] + '/git/ImageD11')
import ImageD11.sinograms.dataset
import ImageD11.sinograms.properties
import ImageD11.sinograms.roi_iradon
import ImageD11.blobcorrector
import ImageD11.nbGui.nb_utils
import ImageD11.columnfile
import ImageD11.unitcell

In [ ]:
!pwd

In [ ]:
dsname="/data/visitor/ihma423/id11/20231205/PROCESSED_DATA/SparsePixels_NewMask/ds_S9_deformed_sliceZ_7.h5"
# To Do: the next four filenames should be stored in the dataset
pkname="/data/visitor/ihma423/id11/20231205/PROCESSED_DATA/SparsePixels_NewMask/pks_S9_deformed_sliceZ_7.h5"
sparsename="/data/visitor/ihma423/id11/20231205/PROCESSED_DATA/SparsePixels_NewMask/S9_deformed_sliceZ_7_sparse.h5"
parname= "/data/visitor/ihma423/id11/20231205/PROCESSED_DATA/S7_deformed/Al_fcc_2.par"
dxfile = "/data/id11/nanoscope/Eiger/e2dx_E-08-0144_20240205.edf"
dyfile = "/data/id11/nanoscope/Eiger/e2dy_E-08-0144_20240205.edf"

# OUTPUTNAME
shapename = "/data/visitor/ihma423/id11/20231205/PROCESSED_DATA/SparsePixels_NewMask/shapes_S9_deformed_sliceZ_7.h5"

In [ ]:
# Load the dataset
ds = ImageD11.sinograms.dataset.load(dsname)

# To Do: tocolf or getpeaks should be a dataset method
pkst = ImageD11.sinograms.properties.pks_table.load(pkname)
cf = ImageD11.nbGui.nb_utils.tocolf( pkst.pk2d(ds.omega, ds.dty), parname, dxfile, dyfile )

In [ ]:
# To normalise data below
with h5py.File(sparsename,'r') as hin:
    monitor = ds.sinohist(np.array([hin[scan]['measurement/fpico6'] for scan in ds.scans]))

In [ ]:
pl.imshow(monitor.T,aspect='auto')
pl.colorbar()

In [ ]:
# Select peaks belonging to a unit cell and below dsmax
dstol = 0.005
dsmax = 1.5

uc = ImageD11.unitcell.unitcell_from_parameters(cf.parameters)
uc.makerings( cf.ds.max() )
la = cf.ds < dsmax
m = np.zeros_like(la)
for d in uc.ringds:
    m |= abs( cf.ds - d ) < dstol
m = ~m & la
ringmask=m

In [ ]:
# Show what was selected and what was not
f,a = pl.subplots(2,1)
a[0].hist2d( cf.ds, np.log( cf.sum_intensity ), bins=(5000, 128), norm=pl.matplotlib.colors.LogNorm())
a[1].hist2d( cf.ds[m], np.log( cf.sum_intensity[m] ), bins=(5000, 128), norm=pl.matplotlib.colors.LogNorm());

In [ ]:
# A series of different functions to try:
funcs = { 'log': np.log, 
          #'sqrt': np.sqrt, 
          #'x^{1/3}' : lambda x: pow(x,1/3),
          'linear' : lambda x: x,
        }

In [ ]:
def circlemask(n):
    """ mask the central circle for tomo recon """
    i,j = np.mgrid[0:n,0:n]
    r = i.max()//2
    i -= r
    j -= r
    m = i*i+j*j < r*r
    return m

In [ ]:
#test case
sino = ds.sinohist( np.log( cf.sum_intensity ) , cf.omega, cf.dty )

In [ ]:
offset=0.0
output_size=331
pl.imshow( circlemask(output_size)*
    ImageD11.sinograms.roi_iradon.iradon( (sino/monitor).T, ds.obincens, output_size=output_size,
                                                       projection_shifts=np.full_like(sino_ppt.T, offset) ),
         vmin=0.
         )

In [ ]:
results = {}

nt = ImageD11.cImageD11.cores_available()
nouter=10
niter = 10     # for mlem 10x10 = 100
offset = 0.0    # dty zero was not well aligned
cm = circlemask(output_size)

for fname in funcs:
    fun = funcs[fname]
    print(fname)
    sino_all = ds.sinohist( fun( cf.sum_intensity[~m] ) , cf.omega[~m], cf.dty[~m] )/monitor
    sino_ppt = ds.sinohist( fun( cf.sum_intensity[m] ) , cf.omega[m], cf.dty[m] )/monitor
    iradon_all = ImageD11.sinograms.roi_iradon.iradon( sino_all.T, ds.obincens, output_size=output_size,
                                                       projection_shifts=np.full_like(sino_all.T, offset) )
    iradon_ppt = ImageD11.sinograms.roi_iradon.iradon( sino_ppt.T, ds.obincens, output_size=output_size,
                                                       projection_shifts=np.full_like(sino_ppt.T, offset) )
    mlem_all = cm.copy()
    mlem_ppt = cm.copy()
    for i in range(nouter):
        mlem_all   = ImageD11.sinograms.roi_iradon.mlem( sino_all.T, ds.obincens, output_size=output_size, niter=niter, workers=nt,
                                                     projection_shifts=np.full_like(sino_all.T, offset),
                                                     startvalue=mlem_all)
        
        mlem_ppt   = ImageD11.sinograms.roi_iradon.mlem( sino_ppt.T, ds.obincens, output_size=output_size, niter=niter, workers=nt,
                                                    projection_shifts=np.full_like(sino_ppt.T, offset),
                                                    startvalue=mlem_ppt)
        print('.',end='')
    results[ fname ] = {
        'sino_all':sino_all,
        'sino_ppt':sino_ppt,
        'iradon_all':iradon_all,
        'iradon_ppt':iradon_ppt,
        'mlem_all':mlem_all,
        'mlem_ppt':mlem_ppt}

In [ ]:
with h5py.File(shapename,"a") as hout:
    for name in results:
        print(name)
        grp = hout.require_group( name )
        for arname in results[name]:
            ar = results[name][arname]
            ds = grp.require_dataset( arname, shape=ar.shape, dtype=ar.dtype)     
            ds[:] = ar

In [ ]:
def imr(ax,ar,cut=0.25, **kwds):
    return ax.imshow( ar, vmin=0, vmax=ar.max()*cut, cmap='gray_r',**kwds)

for fname in results:
    r = results[fname]
    f, (a,b) = pl.subplots(2,3,figsize=(20,10))
    imr(a[0], r['sino_all'].T, cut=1 , aspect='auto')
    a[0].set(title=f'sinogram( {fname} ( all peaks )) ', xlabel='omega', ylabel='dty')
    imr(a[1], r['iradon_all'],cut=1)
    a[1].set(title=f'iradon( {fname} ( all peaks )) ', xlabel='x', ylabel='y')
    m = circlemask(output_size)
    imr(a[2],r['mlem_all']*m, cut=1 )
    a[2].set(title=f'mlem( {fname} ( all peaks )) ', xlabel='x', ylabel='y')

    imr(b[0], r['sino_ppt'].T, cut=0.2 , aspect='auto')
    b[0].set(title=f'sinogram( {fname} ( ppt peaks )) ', xlabel='omega', ylabel='dty')
    imr(b[1],r['iradon_ppt'], cut=0.2 )
    b[1].set(title=f'iradon( {fname} ( ppt peaks )) ', xlabel='x', ylabel='y')
    imr(b[2],r['mlem_ppt'], cut=0.2) 
    b[2].set(title=f'mlem( {fname} ( ppt peaks )) ', xlabel='x', ylabel='y')
    pl.show()